# Random Forest Regressor 

## Mục tiêu
- Xây dựng mô hình Random Forest Regressor để dự đoán tuổi thọ trung bình
- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`
- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation
- Đánh giá mô hình trên tập train
- Lưu mô hình đã huấn luyện 


## Giới thiệu

Là tập hợp nhiều cây quyết định độc lập (bagging ensemble).

### **Ưu điểm:**

- Giảm overfitting so với Decision Tree.

- Ổn định, mạnh mẽ.

- Tự động đánh giá tầm quan trọng của biến.

- Xử lý tốt dữ liệu lớn.

### **Nhược điểm:**

- Không dễ diễn giải như 1 cây duy nhất.

- Cần nhiều tài nguyên tính toán.

## Bước 1 - Import các thư viện cần thiết

In [1]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý và chia sẵn thành train/validation/test.

In [2]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [3]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình Random Forest 

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Random Forest.


In [4]:
from scipy.stats import randint, uniform

# Định nghĩa không gian siêu tham số để tìm kiếm
param_distributions = {
    "n_estimators": randint(200, 800),            
    "max_depth": randint(3, 20),               
    "max_features": uniform(0.3, 0.7),         
    "min_samples_split": randint(2, 20),         
    "min_samples_leaf": randint(1, 10),       
    "bootstrap": [True, False],
}

# Tạo RandomizedSearchCV
# Do số lượng siêu tham số cần tìm là rất lớn nên ta chỉ áp dụng RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_distributions,
    cv=5,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    n_iter=60,
    random_state=42
)

# Thực hiện tìm kiếm
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


[CV] END bootstrap=False, max_depth=3, max_features=0.5129695700716763, min_samples_leaf=6, min_samples_split=13, n_estimators=760; total time=   4.2s
[CV] END bootstrap=False, max_depth=4, max_features=0.8053991405867773, min_samples_leaf=6, min_samples_split=3, n_estimators=391; total time=   4.2s
[CV] END bootstrap=False, max_depth=4, max_features=0.8053991405867773, min_samples_leaf=6, min_samples_split=3, n_estimators=391; total time=   4.2s
[CV] END bootstrap=True, max_depth=13, max_features=0.621474224376107, min_samples_leaf=5, min_samples_split=5, n_estimators=330; total time=   4.3s
[CV] END bootstrap=True, max_depth=13, max_features=0.621474224376107, min_samples_leaf=5, min_samples_split=5, n_estimators=330; total time=   4.4s
[CV] END bootstrap=False, max_depth=3, max_features=0.5129695700716763, min_samples_leaf=6, min_samples_split=13, n_estimators=760; total time=   4.3s
[CV] END bootstrap=False, max_depth=4, max_features=0.8053991405867773, min_samples_leaf=6, min_samp

,estimator,RandomForestR...ndom_state=42)
,param_distributions,"{'bootstrap': [True, False], 'max_depth': <scipy.stats....x75d623916cc0>, 'max_features': <scipy.stats....x75d623916e10>, 'min_samples_leaf': <scipy.stats....x75d6239168a0>, ...}"
,n_iter,60
,scoring,'neg_root_mean_squared_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất.

In [5]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_rf = random_search.best_estimator_
best_params = random_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây: {best_rf.n_estimators}")
print(f"Số đặc trưng xem xét mỗi lần chia: {best_rf.max_features}")
print(f"Bootstrap sampling: {best_rf.bootstrap}")

SIÊU THAM SỐ TỐI ƯU
  bootstrap           : False
  max_depth           : 17
  max_features        : 0.42135525745544045
  min_samples_leaf    : 1
  min_samples_split   : 5
  n_estimators        : 761

Số lượng cây: 761
Số đặc trưng xem xét mỗi lần chia: 0.42135525745544045
Bootstrap sampling: False


### 4.3. Đánh giá mô hình trên tập train

In [14]:
# Dự đoán trên tập train
y_pred = best_rf.predict(X_train)

# Tính toán các metrics cho tập train
train_mae = mean_absolute_error(y_train, y_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred))
train_r2 = r2_score(y_train, y_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: Random Forest:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss: {train_mae:.3f}")
print(f"R2 score: {train_r2:.3f}")

KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: Random Forest:
RMSE loss: 0.380
MAE loss: 0.200
R2 score: 0.998


### 4.4. Lưu mô hình 

In [7]:
import joblib
import os

# Tạo thư mục lưu model nếu chưa có
os.makedirs('../models/4_random_forest', exist_ok=True)

# Lưu mô hình
joblib.dump(best_rf, "../models/4_random_forest/random_forest.pkl.gz", compress=('gzip', 6))

['../models/4_random_forest/random_forest.pkl.gz']

## Kết luận

### Tổng kết:
- Mô hình Random Forest đã được xây dựng và tối ưu hóa thành công
- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu
- Mô hình được đánh giá dựa trên 3 metrics: MAE, RMSE và R2